In [12]:
from fastai.vision.all import *
from random import randint
from ml_utils import *
import os

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
clips_df = pd.read_csv('data/labels.csv')

# dropping dangling columns
clips_df = clips_df.loc[:, ~clips_df.columns.str.contains('^Unnamed')]

def clean_labels(c): return str([(i).strip().replace(" ", "_").replace("-", "_").replace("/", "_") for i in re.sub(r"[\[\]]",'',c).split(',')]).replace("'","").strip("'").strip("[").strip("]").replace(" ","")
clips_df['tools_present'] = clips_df['tools_present'].apply(clean_labels)
clips_df.head()

,clip_name,tools_present
0,clip_000000,"needle_driver,nan,needle_driver,cadiere_forceps"
1,clip_000001,"needle_driver,nan,needle_driver,cadiere_forceps"
2,clip_000002,"needle_driver,nan,needle_driver,cadiere_forceps"
3,clip_000003,"needle_driver,nan,needle_driver,cadiere_forceps"
4,clip_000004,"needle_driver,nan,needle_driver,cadiere_forceps"


In [15]:
df=clips_df.groupby('tools_present').nth(0)
len(df)

153

In [16]:
src=Path('data/train_images_crop')
dst=Path('data/testing/images')

In [ ]:
for i in get_image_files(dst): os.remove(i) 

In [17]:
for clip in df.clip_name:
    fs = get_image_files(src/clip)
    if len(fs)>0:
        fno=randint(0,len(fs)-1)
        im=PILImage.create(fs[fno])
        fn=str(dst)+'/'+clip+'_'+fs[fno].name
        im.save(fn)
    

In [18]:
ensem_path=Path("test/algorithm/cls")

In [19]:
ensem_learner=[load_learner(m, cpu=True) for m in ensem_path.ls() if m.suffix=='.pkl']

In [20]:
fs=get_image_files(dst);len(fs)

153

In [21]:
tta_res=[]
prs_items=[]
for learn in ensem_learner:
    tta_res.append(learn.tta(dl=learn.dls.test_dl(fs)))
    if len(prs_items)<1:
        prs_items=learn.dl.items

tta_prs=first(zip(*tta_res))
tta_prs=torch.stack(tta_prs)

lbls=[]
for i in range(len(c)):
    arm_preds = tta_prs[:,:,cfg(i):cfg(i+1)].mean(0);
    arm_idxs = arm_preds.argmax(dim=1)
    arm_vocab = np.array(vocab[i])
    lbls.append(arm_vocab[arm_idxs])


In [22]:
# data=[]
# for lbl,f in zip(lbls,prs_items):
#     l1,l2,l3,l4=lbls
#     data.append((f,l1+','+l2+','+l3+','+l4))
# df = pd.DataFrame(data,columns=['image_id','PR_labels'])
# df

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U30'), dtype('<U1')) -> None

In [23]:
data=[]
for l1,l2,l3,l4,f in zip(lbls[0],lbls[1],lbls[2],lbls[3],prs_items):
    print((f,l1+','+l2+','+l3+','+l4))
    data.append((f,l1+','+l2+','+l3+','+l4))
    data.append((f,l1+','+l2+','+l3+','+l4))

(Path('data/testing/images/clip_011250_01380.jpg'), 'cadiere_forceps,force_bipolar,nan,permanent_cautery_hook_spatula')
(Path('data/testing/images/clip_001104_01770.jpg'), 'bipolar_forceps,nan,needle_driver,grasping_retractor')
(Path('data/testing/images/clip_023116_00735.jpg'), 'force_bipolar,nan,clip_applier,prograsp_forceps')
(Path('data/testing/images/clip_002795_01335.jpg'), 'bipolar_forceps,nan,vessel_sealer,grasping_retractor')
(Path('data/testing/images/clip_005961_01695.jpg'), 'tip_up_fenestrated_grasper,cadiere_forceps,nan,vessel_sealer')
(Path('data/testing/images/clip_000177_01110.jpg'), 'grasping_retractor,bipolar_forceps,nan,monopolar_curved_scissor')
(Path('data/testing/images/clip_013016_01605.jpg'), 'force_bipolar,nan,cadiere_forceps,stapler')
(Path('data/testing/images/clip_017382_01290.jpg'), 'needle_driver,nan,needle_driver,force_bipolar')
(Path('data/testing/images/clip_013044_00075.jpg'), 'bipolar_forceps,nan,clip_applier,prograsp_forceps')
(Path('data/testing/ima

In [24]:
clips_df.set_index('clip_name',inplace=True,drop=True)

In [25]:
def get_GT_labels(f):
    c=str(Path(f).name).rpartition('_')[0]
    return clips_df.loc[c,'tools_present']

In [26]:
# df = pd.DataFrame(data,columns=['image_id','PR_labels'])
# df

In [27]:
df = pd.DataFrame(data,columns=['image_id','PR_labels'])
df['GT_labels']=df['image_id'].map(get_GT_labels) 
df['accurate_tf'] = (df['PR_labels']==df['GT_labels'])
df

,image_id,PR_labels,GT_labels,accurate_tf
0,data/testing/images/clip_011250_01380.jpg,"cadiere_forceps,force_bipolar,nan,permanent_cautery_hook_spatula","cadiere_forceps,force_bipolar,nan,permanent_cautery_hook_spatula",True
1,data/testing/images/clip_011250_01380.jpg,"cadiere_forceps,force_bipolar,nan,permanent_cautery_hook_spatula","cadiere_forceps,force_bipolar,nan,permanent_cautery_hook_spatula",True
2,data/testing/images/clip_001104_01770.jpg,"bipolar_forceps,nan,needle_driver,grasping_retractor","bipolar_forceps,nan,needle_driver,grasping_retractor",True
3,data/testing/images/clip_001104_01770.jpg,"bipolar_forceps,nan,needle_driver,grasping_retractor","bipolar_forceps,nan,needle_driver,grasping_retractor",True
4,data/testing/images/clip_023116_00735.jpg,"force_bipolar,nan,clip_applier,prograsp_forceps","force_bipolar,nan,clip_applier,prograsp_forceps",True
5,data/testing/images/clip_023116_00735.jpg,"force_bipolar,nan,clip_applier,prograsp_forceps","force_bipolar,nan,clip_applier,prograsp_forceps",True
6,data/testing/images/clip_002795_01335.jpg,"bipolar_forceps,nan,vessel_sealer,grasping_retractor","bipolar_forceps,nan,vessel_sealer,grasping_retractor",True
7,data/testing/images/clip_002795_01335.jpg,"bipolar_forceps,nan,vessel_sealer,grasping_retractor","bipolar_forceps,nan,vessel_sealer,grasping_retractor",True
8,data/testing/images/clip_005961_01695.jpg,"tip_up_fenestrated_grasper,cadiere_forceps,nan,vessel_sealer","tip_up_fenestrated_grasper,cadiere_forceps,nan,vessel_sealer",True
9,data/testing/images/clip_005961_01695.jpg,"tip_up_fenestrated_grasper,cadiere_forceps,nan,vessel_sealer","tip_up_fenestrated_grasper,cadiere_forceps,nan,vessel_sealer",True
